In [1]:
from sympy import *
import math
import numpy as np

### 11.2

In [2]:
h = Symbol('h')
A = Symbol('A')
ф_f = Symbol('ф_f')

In [3]:
km = Matrix([[h*A, 0],
           [0,0]])

In [4]:
km

Matrix([
[A*h, 0],
[  0, 0]])

In [5]:
fs = Matrix([h*A*ф_f,0])

In [6]:
fs

Matrix([
[A*h*ф_f],
[      0]])

### 11.3

In [7]:
def FinAssemble(K,k,i,j):
    K[i,i] = K[i,i] + k[0,0]
    K[i,j] = K[i,j] + k[0,1]
    K[j,i] = K[j,i] + k[1,0]
    K[j,j] = K[j,j] + k[1,1]
    return K

In [8]:
def ElementForceQ(h,P,L,ф_f):
    return h*P*L*ф_f/2*np.array([1,1])

In [9]:
def ElementForceS(h,P,L,ф_f,A):
    return h*P*L*ф_f/2*np.array([1,1])+h*A*ф_f*np.array([0,1])

In [10]:
def ForceAssemble(F,f,i,j):
    F[i]=F[i]+f[0]
    F[j]=F[j]+f[1]
    return F

In [11]:
def ElementStiffness(kc,A,L,P,h):
    return kc*A/L*np.matrix([[1,-1], [-1,1]]) + h*P*L/6*np.matrix([[2,1], [1,2]])

In [12]:
def ElementStiffnessM(kc,A,L,P,h):
    return (kc*A/L*np.matrix([[1,-1], [-1,1]]) 
            + h*P*L/6*np.matrix([[2,1], [1,2]])
            +np.matrix([[0,0], 
                        [0,h*A]]))

In [13]:
def FEA_ф(inp, bc_node, bc_val):
    K=np.zeros(shape=(len(inp)+1, len(inp)+1))
    
    F=np.zeros(shape=(len(inp)+1,))
    
    max_coord=0
    for t, el in enumerate(inp):
        Xi, Xj, kc, A, P, h, ф_f, i, j = el
        max_coord=max(max_coord,i,j)
        if t==len(inp)-1:
            k=ElementStiffnessM(kc,A,Xj-Xi,P,h)
            f=ElementForceS(h,P,Xj-Xi,ф_f,A)
        else:
            k=ElementStiffness(kc,A,Xj-Xi,P,h)
            f=ElementForceQ(h,P,Xj-Xi,ф_f)
        K=FinAssemble(K, k, i, j)
        F=ForceAssemble(F,f,i,j)
        
#     https://github.com/CALFEM/calfem-matlab/blob/master/fem/solveq.m
#     https://www.byggmek.lth.se/fileadmin/byggnadsmekanik/publications/tvsm5000/web5167.pdf p.35  

#     F = np.zeros(shape =(6, 1)) 
#     F[2] = 10
     
     
#     K=np.matrix([[ 25. , -25. , 0. , 0. , 0. , 0. ],
#         [-25. , 49.3, -24.3, 0. , 0. , 0. ],
#         [ 0. , -24.3, 24.7, -0.4, 0. , 0. ],
#         [ 0. , 0. , -0.4, 17.4, -17. , 0. ],
#         [ 0. , 0. , 0. , -17. , 24.7, -7.7],
#         [ 0. , 0. , 0. , 0. , -7.7, 7.7]])
    
#     bc = np.array([[0, 5],
#            [-17.0, 20.0]]).astype(int).T
    
    bc=np.array([bc_node, 
                bc_val]).T

    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))


    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    s=np.linalg.solve(K[fdof,:][:,fdof],(F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T)
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
    Q=np.dot(K,d).T-F
    
    return d, Q

In [14]:
kc=3
A=4
P=10
h=0.1
ф_f=20

In [15]:
inp = [[0,1,kc,A,P,h,ф_f,0,1],
       [1,3,kc,A,P,h,ф_f,1,2],
       [3,5,kc,A,P,h,ф_f,2,3],
       [5,8,kc,A,P,h,ф_f,3,4]]

In [16]:
d,Q = FEA_ф(inp, [0],[80.])

In [17]:
d

array([80.        , 65.04539384, 45.74043816, 35.52034301, 30.05948158])

In [18]:
Q

array([ 2.06962840e+02,  0.00000000e+00, -1.13686838e-13,  7.10542736e-14,
       -1.42108547e-14])

In [19]:
inp = [[0,2,kc,A,P,h,ф_f,0,1],
       [2,4,kc,A,P,h,ф_f,1,2],
       [4,6,kc,A,P,h,ф_f,2,3],
       [6,8,kc,A,P,h,ф_f,3,4]]

In [20]:
d,Q = FEA_ф(inp, [0],[80.])

In [21]:
d

array([80.        , 53.94556119, 39.87190868, 32.811871  , 30.27367014])

### 11.5

In [22]:
kc=2
D=1
A=math.pi*D**2/4
P=math.pi*D
h=0.2
ф_f=10

In [23]:
inp = [[0,1,kc,A,P,h,ф_f,0,1],
       [1,3,kc,A,P,h,ф_f,1,2],
       [3,6,kc,A,P,h,ф_f,2,3]]

In [24]:
d,Q = FEA_ф(inp, [0],[50.])

In [25]:
d

array([50.        , 30.63667176, 15.11729913, 10.81876786])

In [26]:
# TODO - the answer is not the same as in the book

### 11.8

In [27]:
kc=0.5

In [28]:
ф_f_left=5
ф_f_right=20
h_left=1.5
h_right=0.05

In [29]:
A=1

In [30]:
L=4

In [31]:
K=(kc*A/L*np.matrix([[1,-1], [-1,1]]) 
   +np.matrix([[h_left*A,0], [0,0]])
   +np.matrix([[0,0], [0,h_right*A]]))

In [32]:
K

matrix([[ 1.625, -0.125],
        [-0.125,  0.175]])

In [33]:
F=h_left*A*ф_f_left*np.array([1,0])+h_right*A*ф_f_right*np.array([0,1])

In [34]:
F

array([7.5, 1. ])

In [35]:
bc=np.array([[1], [30]]).T

nd, nd=K.shape
fdof=np.array([i for i in range(nd)]).T
d=np.zeros(shape=(len(fdof),))
Q=np.zeros(shape=(len(fdof),))


pdof=bc[:,0].astype(int)
dp=bc[:,1]
fdof=np.delete(fdof, pdof, 0)
s=np.linalg.solve(K[fdof,:][:,fdof],(F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T)
d[pdof]=dp
d[fdof]=s.reshape(-1,)

In [36]:
d

array([ 6.92307692, 30.        ])

In [37]:
# TODO - the answer is not the same as in the book

### 11.9

In [38]:
# check Segerlind 1st edition for theoretical derivations

In [39]:
Xi=6
Yi=2
Xj=10
Yj=1
Xk=8
Yk=6
h=0.2
ф_f=20
Q=5
kx=2
ky=2

In [40]:
A=0.5*np.linalg.det(np.matrix(  
            [[1,Xi,Yi],
            [1,Xj,Yj],
            [1,Xk,Yk]]))

In [41]:
bi=Yj-Yk
bj=Yk-Yi
bk=Yi-Yj
ci=Xk-Xj
cj=Xi-Xk
ck=Xj-Xi

In [42]:
kd_1=kx/(4*A)*np.matrix([[bi**2, bi*bj, bi*bk],
                        [bi*bj, bj**2, bj*bk],
                        [bi*bk, bj*bk, bk**2]])

kd_2=ky/(4*A)*np.matrix([[ci**2, ci*cj, ci*ck],
                        [ci*cj, cj**2, cj*ck],
                        [ci*ck, cj*ck, ck**2]])

In [43]:
kd = kd_1 + kd_2  

In [44]:
Lij = math.sqrt((Xi - Xj)**2 + (Yi - Yj)**2)

In [45]:
kg = h*Lij/6*np.matrix([[2, 1, 0],
                      [1, 2, 0],
                      [0, 0, 0]])

In [46]:
k=kd+kg

In [47]:
k

matrix([[ 1.88598482, -0.75145203, -0.72222222],
        [-0.75145203,  1.38598482, -0.22222222],
        [-0.72222222, -0.22222222,  0.94444444]])

In [48]:
f=h*ф_f*Lij/2*np.array([1,1,0])+Q/3*np.array([1,1,1])

In [49]:
f

array([9.91287792, 9.91287792, 1.66666667])

In [50]:
# TODO - the answer is not the same as in the book (for f)

In [51]:
d=np.linalg.lstsq(k, f, rcond=None)[0]

In [52]:
d

array([26.25902701, 25.86775424, 27.93166871])

### 11.11

In [53]:
# check Segerlind 1st edition for theoretical derivations

In [54]:
Xi = -2
Yi = -2
Xj = 4
Yj = 0
Xk = 0
Yk = 4
h = 0.2
ф_f = 15
kx = 2
ky = 2

In [55]:
Lki = math.sqrt((Xk - Xi)**2 + (Yk - Yi)**2)
# Lij = math.sqrt((Xi - Xj)**2 + (Yi - Yj)**2)
# Ljk = math.sqrt((Xj - Xk)**2 + (Yj - Yk)**2)

In [56]:
A=0.5*np.linalg.det(np.matrix(  
            [[1,Xi,Yi],
            [1,Xj,Yj],
            [1,Xk,Yk]]))

In [57]:
A

16.0

In [58]:
bi=Yj-Yk
bj=Yk-Yi
bk=Yi-Yj
ci=Xk-Xj
cj=Xi-Xk
ck=Xj-Xi

In [59]:
kd_1=kx/(4*A)*np.matrix([[bi**2, bi*bj, bi*bk],
                        [bi*bj, bj**2, bj*bk],
                        [bi*bk, bj*bk, bk**2]])

kd_2=ky/(4*A)*np.matrix([[ci**2, ci*cj, ci*ck],
                        [ci*cj, cj**2, cj*ck],
                        [ci*ck, cj*ck, ck**2]])

In [60]:
kd = kd_1 + kd_2  

In [61]:
kg = h*Lki/6*np.matrix([[2, 0, 1],
                        [0, 0, 0],
                        [1, 0, 2]])

In [62]:
k=kd+kg

In [63]:
k

matrix([[ 1.42163702, -0.5       , -0.28918149],
        [-0.5       ,  1.25      , -0.75      ],
        [-0.28918149, -0.75      ,  1.67163702]])

In [64]:
f=h*ф_f*Lki/2*np.array([1,0,1]) 

In [65]:
f

array([9.48683298, 0.        , 9.48683298])

In [66]:
# TODO - the answer is not the same as in the book 

In [67]:
d=np.linalg.lstsq(k, f, rcond=None)[0]

In [68]:
d

array([15., 15., 15.])